# NLP Critique de film

## Importation des bibliothèques

In [1]:
import requests
import lxml.html as lh
import pandas as pd

## Scrapping

In [2]:
page = requests.get("https://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/")
doc = lh.fromstring(page.content)

com = doc.xpath('//div[@class="content-txt review-card-content"]')
note = doc.xpath('//span[@class="stareval-note"]')

In [3]:
# for i in range(len(note)):
#     print(note[i].text_content().strip() + "\n" + \
#           com[i].text_content().strip() + "\n")

In [4]:
# URL
url = "https://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/"
# URI-SUFFIX
uri_pages = '?page='
# Number of pages to read
nbPages = 100
# XPath content to collect
tags = ['//span[@class="stareval-note"]', \
        '//div[@class="content-txt review-card-content"]']

cols = ["Note", \
        "Commentaire"]

urls = ["https://www.allocine.fr/film/fichefilm-143692/critiques/spectateurs/", \
        "https://www.allocine.fr/film/fichefilm-281203/critiques/spectateurs/"]

In [5]:
page = requests.get(url)
doc = lh.fromstring(page.content)

In [6]:
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)

    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))

    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[cols[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)

    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
    
    return liste

def getPages(_nbPages, _urls):
    liste_finale = pd.DataFrame()
    for _url in _urls:
        for i in range (_nbPages):
            liste = getPage(_url + uri_pages + str(i+1))
            liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
        print(liste_finale.shape)
    return liste_finale

In [7]:
getPage(url).shape

(15, 3)

In [8]:
list_totale = getPages(nbPages, urls)

(1485, 3)
(1558, 3)


In [10]:
list_totale

,Note,Commentaire,key
0,"5,0","Dans la catégorie des blockbusters, on a rarem...",0
1,"4,5","C'est l'après dark knight nolan, nous livre un...",1
2,"5,0","En 2010, Christopher Nolan, réalisateur de fil...",2
3,"4,0",Assurément l'un des meilleurs films de l'année...,3
4,"5,0",Inception est un très grand film. Les acteurs ...,4
...,...,...,...
1553,"3,0","léger et sans prise de tête, le film permet de...",8
1554,"0,5",Films vraiment pour les enfants je me suis fai...,9
1555,"5,0","Ce film etait juste ouffissime , j'ai vraiment...",10
1556,"1,0","Déconseillé aux enfants de plus de 7 ans, et e...",11


## Préparation des données

In [11]:
df = list_totale.copy()

In [12]:
df["Commentaire"] = df["Commentaire"].replace("[^\w\s]", " ", regex=True).str.replace('\d+', '', regex=True).str.lower()
df.head()

,Note,Commentaire,key
0,"5,0",dans la catégorie des blockbusters on a rarem...,0
1,"4,5",c est l après dark knight nolan nous livre un...,1
2,"5,0",en christopher nolan réalisateur de films c...,2
3,"4,0",assurément l un des meilleurs films de l année...,3
4,"5,0",inception est un très grand film les acteurs ...,4


### Soustraction de stopwords 

In [13]:
from stop_words import get_stop_words

In [14]:
stopwords = get_stop_words('french')
print(stopwords)

['a', 'ai', 'aie', 'aient', 'aies', 'ait', 'alors', 'as', 'au', 'aucun', 'aura', 'aurai', 'auraient', 'aurais', 'aurait', 'auras', 'aurez', 'auriez', 'aurions', 'aurons', 'auront', 'aussi', 'autre', 'aux', 'avaient', 'avais', 'avait', 'avant', 'avec', 'avez', 'aviez', 'avions', 'avoir', 'avons', 'ayant', 'ayez', 'ayons', 'bon', 'car', 'ce', 'ceci', 'cela', 'ces', 'cet', 'cette', 'ceux', 'chaque', 'ci', 'comme', 'comment', 'd', 'dans', 'de', 'dedans', 'dehors', 'depuis', 'des', 'deux', 'devoir', 'devrait', 'devrez', 'devriez', 'devrions', 'devrons', 'devront', 'dois', 'doit', 'donc', 'dos', 'droite', 'du', 'dès', 'début', 'dù', 'elle', 'elles', 'en', 'encore', 'es', 'est', 'et', 'eu', 'eue', 'eues', 'eurent', 'eus', 'eusse', 'eussent', 'eusses', 'eussiez', 'eussions', 'eut', 'eux', 'eûmes', 'eût', 'eûtes', 'faire', 'fais', 'faisez', 'fait', 'faites', 'fois', 'font', 'force', 'furent', 'fus', 'fusse', 'fussent', 'fusses', 'fussiez', 'fussions', 'fut', 'fûmes', 'fût', 'fûtes', 'haut', 'ho

In [15]:
stopwords.append("c")

In [16]:
df['sans_sw'] = df['Commentaire'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df.head()

,Note,Commentaire,key,sans_sw
0,"5,0",dans la catégorie des blockbusters on a rarem...,0,catégorie blockbusters rarement mieux mise scè...
1,"4,5",c est l après dark knight nolan nous livre un...,1,après dark knight nolan livre divertissements ...
2,"5,0",en christopher nolan réalisateur de films c...,2,christopher nolan réalisateur films cultes aut...
3,"4,0",assurément l un des meilleurs films de l année...,3,assurément meilleurs films année jour niveau n...
4,"5,0",inception est un très grand film les acteurs ...,4,inception grand film acteurs excellents scénar...


### Etape de LEM

In [17]:
import spacy

In [18]:
nlp = spacy.load('fr_core_news_md')

# doc = nlp(u"voudrais était non animaux yeux dors couvre.")
# for token in doc:
#    print(token, token.lemma_)

In [111]:
# df["lem"] = df["sans_sw"].apply(lambda x: [y.lemma_ for y in nlp(x)])
# df.head()

,Note,Commentaire,key,sans_sw,lem
0,"5,0",après le chef d oeuvre super héroïque the dark...,0,après chef oeuvre super héroïque the dark knig...,"[après, chef, oeuvre, super, héroïque, the, da..."
1,"5,0",c est fou ce qu on aime détester christopher n...,1,fou aime détester christopher nolan plus films...,"[fou, aime, détester, christopher, nolan, plus..."
2,"5,0",chef d œuvre le film est absolument parfait ...,2,chef œuvre film absolument parfait acteurs exc...,"[chef, œuvre, film, absolument, parfait, acteu..."
3,"5,0",un film aussi novateur que complexe dont la m...,3,film novateur complexe dont mise scène touche ...,"[film, novateur, complexe, dont, mettre, scène..."
4,"3,5",le meilleur blockbuster de a pour thème la dé...,4,meilleur blockbuster thème délicate différenci...,"[meilleur, blockbuster, thème, délicat, différ..."


In [19]:
df['lem'] = df["sans_sw"].apply(lambda x: ' '.join(str(y.lemma_) for y in nlp(x)))
df.head()

,Note,Commentaire,key,sans_sw,lem
0,"5,0",dans la catégorie des blockbusters on a rarem...,0,catégorie blockbusters rarement mieux mise scè...,catégorie blockbuster rarement mieux mise scèn...
1,"4,5",c est l après dark knight nolan nous livre un...,1,après dark knight nolan livre divertissements ...,après dark knight nolan livre divertissement d...
2,"5,0",en christopher nolan réalisateur de films c...,2,christopher nolan réalisateur films cultes aut...,christopher nolan réalisateur film culte autre...
3,"4,0",assurément l un des meilleurs films de l année...,3,assurément meilleurs films année jour niveau n...,assurément meilleur film année jour niveau nol...
4,"5,0",inception est un très grand film les acteurs ...,4,inception grand film acteurs excellents scénar...,inception grand film acteur excellent scénario...


### Retrait des accents

In [57]:
df["lem"] = df["lem"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df.head()

,Note,Commentaire,key,sans_sw,lem
0,"5,0",dans la catégorie des blockbusters on a rarem...,0,catégorie blockbusters rarement mieux mise scè...,categorie blockbuster rarement mieux mise scen...
1,"4,5",c est l après dark knight nolan nous livre un...,1,après dark knight nolan livre divertissements ...,apres dark knight nolan livre divertissement d...
2,"5,0",en christopher nolan réalisateur de films c...,2,christopher nolan réalisateur films cultes aut...,christopher nolan realisateur film culte autre...
3,"4,0",assurément l un des meilleurs films de l année...,3,assurément meilleurs films année jour niveau n...,assurement meilleur film annee jour niveau nol...
4,"5,0",inception est un très grand film les acteurs ...,4,inception grand film acteurs excellents scénar...,inception grand film acteur excellent scenario...


#### Si liste de mots

In [58]:
# import unicodedata as ud

In [59]:
# df["lem"] = df["lem"].apply(lambda x: [ud.normalize('NFKD',y).encode('ascii', errors='ignore').decode('utf-8') for y in x])
# df.head()

### Etape de *STEM*

In [60]:
# from nltk.stem.snowball import SnowballStemmer

In [61]:
# stemmer = SnowballStemmer("french")

In [62]:
# df["stemmed"] = df["sans_sw"].str.split().apply(lambda x: [stemmer.stem(y) for y in x])
# df.head()

### Division du jeu de données en train et test

In [63]:
from sklearn.model_selection import train_test_split

In [64]:
x = df["lem"].copy()
y = df["Note"].replace(",", ".", regex=True).astype("float")

In [65]:
y[y<3.0] = 0
y[y>0] = 1

In [66]:
y.value_counts()

1.0    1254
0.0     304
Name: Note, dtype: int64

In [67]:
y.shape

(1558,)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=0)

### Transformation en *Bag of words*

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

In [45]:
cv = CountVectorizer(lowercase=False, min_df=0.1)
cv_matrix = cv.fit_transform(X_train)

print(cv_matrix.shape, "\n", type(cv_matrix))

(1246, 75) 
 <class 'scipy.sparse._csr.csr_matrix'>


In [46]:
df_bow = pd.DataFrame(cv_matrix.toarray(), index = X_train.index, columns = cv.get_feature_names_out())
df_bow.head()

,acteur,action,aller,annee,apres,assez,autre,beaucoup,bien,bon,...,scene,special,spectateur,surtout,temps,toujours,tout,voir,vraiment,zimmer
224,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
302,0,0,0,0,0,0,1,0,0,0,...,0,1,1,0,0,0,1,0,0,0
1108,1,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
218,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
1014,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,2,0,0,0,0
